In [3]:
import os
import json
import numpy as np

In [4]:
def fetch_data():
    with open("./data/train.jsonl", 'r') as f:
        train_data = [json.loads(line) for line in f]

    tag_mapping = {1: "NN", 2: "DT", 3: "JJ", 4: "OT"}

    # Preprocess training data
    train_sentences = []
    train_labels = []
    for entry in train_data:
        tokens = entry['tokens']
        pos_tags = entry['pos_tags']
        chunk_tags = entry['chunk_tags']
        
        train_sentences.append(pos_tags)
        train_labels.append(chunk_tags)

    with open("./data/test.jsonl", 'r') as f:
        test_data = [json.loads(line) for line in f]
    # Preprocess training data
    test_sentences = []
    test_labels = []
    for entry in test_data:
        tokens = entry['tokens']
        pos_tags = entry['pos_tags']
        chunk_tags = entry['chunk_tags']
        
        test_sentences.append(pos_tags)
        test_labels.append(chunk_tags)
    
    return train_sentences, test_sentences,  train_labels, test_labels

X_train, X_test, y_train, y_test= fetch_data()


In [5]:
X_train[0]

[1, 4, 3, 1, 4, 4, 3, 1, 4]

In [6]:
def one_hot_encode(input_list):
    encoded_list = []
    for item in input_list:
        one_hot_vector = np.zeros(4)
        one_hot_vector[item - 1] = 1  # Adjust index to start from 0
        encoded_list.append(one_hot_vector.tolist())
    return np.array(encoded_list)

In [7]:
one_hot_encode(X_train[0])

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]])

In [8]:
np.concatenate([np.array([1,0,0]), np.array([1,0])])

array([1, 0, 0, 1, 0])

In [9]:
def into_ho(X_train):
    X_train_ho = []
    for i in range(len(X_train)):
        X = one_hot_encode(X_train[i])
        temp = []
        for j in range(len(X)):
            if j==0:
                temp.append(np.concatenate([np.array([1.0,0.0,0.0,0.0,0.0]), X[j]]))
            else:
                temp.append((np.concatenate([np.array([0]), X[j-1], X[j]] )))
        X_train_ho.append(np.array(temp))
    return X_train_ho




In [10]:
X_train_ho = into_ho(X_train)
X_test_ho = into_ho(X_test)


In [14]:
# print(X_train[0],"\n",X_train_ho[0])

In [15]:
# X_train_ho[0][0].shape

In [16]:
def sigmoid(x):
    return 1/(1.0+(np.exp(-x)))

In [43]:
np.concatenate([np.array([0]), X_train_ho[0][0]])

array([0., 1., 0., 0., 0., 0., 1., 0., 0., 0.])

In [71]:
class SingleRecurrentPerceptron:
    def __init__(self, vec_len):
          
        # Initialize weights and bias
        self.weights = np.random.randn(vec_len+1)
        self.threshold = np.random.randn(1)
    
    def forward(self, inputs):
        """Input is of shape (Tx, 10) where tx is number of words in that example and 10 is length of vector representation of one word"""
        out= []
        Tx, _ = inputs.shape
        y_prev=0
        for i in range(Tx):
            x = np.concatenate([np.array([y_prev]),inputs[i]])
            x = x.T @ self.weights - self.threshold[0]
            x = sigmoid(x)
            y_prev = x
            out.append(x)
        return np.array(out)
    
    def backward(self, inputs, target):
        prediction = self.forward(inputs)
        prediction = np.array(prediction)  # Ensure prediction is a NumPy array
        target = np.array(target)  # Ensure target is a NumPy array
        target = target[:, np.newaxis]  # Reshape target to match prediction's shape
        inputs = inputs[:, np.newaxis]  # Reshape inputs to match prediction's shape
        print("Target:", target)
        print("Prediction:", prediction)
        print("Inputs:", inputs)
        print("Target shape:", target.shape)
        print("Prediction shape:", prediction.shape)
        print("Inputs shape:", inputs.shape)
        error = target - prediction
        self.weights += self.learning_rate * error * inputs
        self.bias += self.learning_rate * error

        
    def train(self, inputs, targets, epochs):
        for _ in range(epochs):
            for i in range(len(inputs)):
                self.backward(inputs[i], targets[i])
    
    def predict(self, inputs):
        predictions = []
        for input_data in inputs:
            prediction = self.forward(input_data)
            predictions.append(prediction)
        return predictions



In [72]:
model = SingleRecurrentPerceptron(9)

In [74]:
model.forward(X_train_ho[1])

array([0.31393441, 0.11372466])

In [ ]:
tag_mapping = {1: "NN", 2: "DT", 3: "JJ", 4: "OT"}

# Calculate the number of unique POS tags
num_unique_tags = len(tag_mapping)

# Preprocess training data
# Preprocess training data
train_sentences = []
train_labels = []
for entry in train_data:
    tokens = entry['tokens']
    pos_tags = entry['pos_tags']
    chunk_tags = entry['chunk_tags']
    
    # Convert POS tags to one-hot encoded representation
    pos_tags_one_hot = np.zeros((len(pos_tags), num_unique_tags))
    for i, tag in enumerate(pos_tags):
        pos_tags_one_hot[i, tag - 1] = 1  # Subtract 1 to account for 0-based indexing
    
    # Flatten one-hot encoded representation
    flattened_tags = pos_tags_one_hot.flatten()
    
    train_sentences.append(flattened_tags)
    train_labels.append(chunk_tags)


# Initialize and train the single recurrent perceptron
# Initialize and train the single recurrent perceptron
input_size = len(train_sentences[0])  # Get input size from the first sample
output_size = 2  # Binary classification (1 for chunk, 0 for not chunk)
perceptron = SingleRecurrentPerceptron(input_size, output_size)
perceptron.train(train_sentences, train_labels, epochs=10)



# Load test data
with open('test.jsonl', 'r') as f:
    test_data = [json.loads(line) for line in f]

# Preprocess test data
test_sentences = []
test_labels = []
for entry in test_data:
    tokens = entry['tokens']
    pos_tags = entry['pos_tags']
    chunk_tags = entry['chunk_tags']
    
    # Convert POS tags to one-hot encoded representation
    pos_tags_one_hot = np.zeros((len(pos_tags), len(tag_mapping)))
    for i, tag in enumerate(pos_tags):
        pos_tags_one_hot[i, tag - 1] = 1  # Subtract 1 to account for 0-based indexing
    
    test_sentences.append(pos_tags_one_hot)
    test_labels.append(chunk_tags)

# Evaluate the trained perceptron
predictions = perceptron.predict(test_sentences)

# Assuming we have some evaluation function to compute accuracy
# Let's assume a simple accuracy calculation for demonstration
def accuracy(predictions, targets):
    correct = 0
    total = len(predictions)
    for pred, target in zip(predictions, targets):
        pred_labels = [1 if p > 0 else 0 for p in pred]
        if pred_labels == target:
            correct += 1
    return correct / total

acc = accuracy(predictions, test_labels)
print("Accuracy:", acc)